In [35]:
import pandas as pd
from config import g_key
import gmaps
import requests

In [36]:
city_data_df = pd.read_csv('weather_data/cities.csv')

In [37]:
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,San Patricio,US,2022-03-31 13:29:01,28.0170,-97.5169,66.06,61,2,7.14
1,1,Naples,IT,2022-03-31 13:29:01,40.8333,14.2500,62.60,89,75,12.66
2,2,Saint-Philippe,RE,2022-03-31 13:29:02,-21.3585,55.7679,80.19,69,50,16.22
3,3,Norman Wells,CA,2022-03-31 13:29:02,65.2820,-126.8329,-14.76,64,40,14.97
4,4,New Norfolk,AU,2022-03-31 13:29:03,-42.7826,147.0587,54.90,66,46,2.75


In [38]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [39]:
# Percent Humidity
# Assign the locations variable
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
# Figure of heat map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat map of humidity
heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = 200, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_map)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df['Wind Speed']
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed,
                                 dissipating= False, max_intensity= 300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [41]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [42]:
# New dataframe for getting min and max temp
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.count()

City_ID       165
City          165
Country       163
Date          165
Lat           165
Lng           165
Max Temp      165
Humidity      165
Cloudiness    165
Wind Speed    165
dtype: int64

In [43]:
preferred_cities_df = preferred_cities_df.dropna()

In [44]:
preferred_cities_df.count()

City_ID       163
City          163
Country       163
Date          163
Lat           163
Lng           163
Max Temp      163
Humidity      163
Cloudiness    163
Wind Speed    163
dtype: int64

In [45]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,80.19,-21.3585,55.7679,
17,Rikitea,PF,77.79,-23.1203,-134.9692,
20,Kulhudhuffushi,MV,84.22,6.6221,73.0700,
22,Faya,SA,78.91,18.3851,42.4509,
24,Lhokseumawe,ID,80.73,5.1801,97.1507,
37,Darnah,LY,79.36,32.7670,22.6367,
38,Banda Aceh,ID,81.14,5.5577,95.3222,
42,Akyab,MM,80.46,20.1500,92.9000,
47,Atuona,PF,78.28,-9.8000,-139.0333,
58,The Valley,AI,80.10,18.2170,-63.0578,


In [49]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [52]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,80.19,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
17,Rikitea,PF,77.79,-23.1203,-134.9692,People ThankYou
20,Kulhudhuffushi,MV,84.22,6.6221,73.0700,Haajy Guest House 1
22,Faya,SA,78.91,18.3851,42.4509,املاك خالد الشعشوع
24,Lhokseumawe,ID,80.73,5.1801,97.1507,OYO 3278 Penginapan Alfajri


In [54]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [55]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [56]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [57]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))